Аналитика следующая:
* Слабые места (куда уходит больше всего)
    - Топ категорий по расходам
    - Доли категорий
    - Всплески расходов по дням (rule-based + ML anomaly)
* Регулярные платежи
    - Группировка по категориям
    - Проверка на периодичность (weekly, monthly)
    - Итог: список "похоже на подписку/регулярный платёж"
* Экономия и "сколько можно отложить"
    - Политика экономии (правила и модель)
    - Расчёт `potential_savings_total`
    - Разложение экономии по категориям и кластерам
    - Итог: "если снизить X%, то можно отложить Y рублей за период N"

In [55]:
from __future__ import annotations

import re
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import IsolationForest
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler


In [56]:
MCC_MAP = {
    "Топливо, АЗС": [5983, 5541, 5542],
    "Автомойка": [7542],

    "Одежда и обувь": [5691, 5137, 5611, 5661, 5621, 5651],

    "Такси и каршеринг": list(range(3351, 3442)) + [4121, 7512, 3990],

    "Фастфуд": [5814],

    "Электроника и техника": [5722, 5732, 5946],

    "Строительные материалы": [5039, 5198, 5211, 5231, 5251, 5713, 5714, 5712, 5074],
    "Товары для дома": [5200, 5718, 5719, 5021, 5261],

    "Автосервисы и запчасти": [7531, 7534, 7535, 7538, 5511, 5521, 5532, 5533],

    "Салоны красоты и СПА": [7230, 7297, 7298],

    "Спортивные магазины": [5655, 5940, 5941],

    "Фитнес": [7941, 7997],

    "Турагентства": [5962, 4722, 4723, 7032, 7033, 4411],

    "Отели": list(range(3501, 3990)) + list(range(3991, 4000)) + [7011],

    "Авиабилеты и ЖД билеты": [
        *list(range(3000, 3302)), 4511, 4582, 4304, 4415, 4418, 4011, 4112
    ],

    "Аптеки": [5122, 5912],

    "Медицинские услуги": [
        8011, 5976, 8031, 8042, 8043, 8049, 8050, 8062, 8021, 8071
    ],

    "Рестораны и кафе": [5811, 5812, 5813],

    "Театры, музеи и выставки": [7922, 7991, 7996, 7998, 7999],

    "Маркетплейсы": [5262, 5300, 7278],

    "Образование": [8211, 8220, 8241, 8244, 8249, 8299, 8351],

    "Детские товары": [5641, 5945],

    "Косметика": [5977],

    "Книги": [2741, 5192, 5942, 5994],

    "Цветы": [5193, 5992],

    "Кино": [7829, 7832, 7841],

    "Канцтовары": [5111, 5943],

    "Товары для животных": [742, 5995],  # 0742 -> 742

    "Бытовые услуги": [
        7210, 7211, 7216, 7217, 7251, 7349, 7379, 7622, 7623, 7629, 7641, 7692, 7699
    ],

    "Связь, интернет, ТВ": [4899, 4813, 4815, 4821, 4816, 4814],
}


In [57]:
def _find_header_row(df_raw: pd.DataFrame, required_cols: List[str], max_scan: int = 200) -> int:
    """Finds header row index in messy exports.

    Args:
        df_raw: Raw dataframe without header.
        required_cols: Keywords that should appear in header row.
        max_scan: Maximum rows to scan.

    Returns:
        Header row index.

    Raises:
        ValueError: If header row can't be detected.
    """
    best_i, best_score = 0, -1
    for i in range(min(len(df_raw), max_scan)):
        row = df_raw.iloc[i].astype(str).str.lower().tolist()
        score = 0
        for col_kw in required_cols:
            if any(col_kw.lower() in cell for cell in row):
                score += 1
        if score > best_score:
            best_score = score
            best_i = i
        if score >= max(1, len(required_cols) - 1):
            return i
    if best_score <= 0:
        raise ValueError("Не удалось найти строку заголовков в файле.")
    return best_i


def load_bank_file(path: str) -> pd.DataFrame:
    """Loads bank export from CSV or XLSX.

    Args:
        path: Path to file.

    Returns:
        Raw dataframe as read from file.
    """
    if path.lower().endswith(".csv"):
        # банки часто в cp1251 / utf-8-sig
        for enc in ("utf-8-sig", "cp1251", "utf-8"):
            try:
                return pd.read_csv(path, encoding=enc)
            except Exception:
                continue
        return pd.read_csv(path)  # fallback
    elif path.lower().endswith((".xlsx", ".xls")):
        df_raw = pd.read_excel(path, header=None)
        header_row = _find_header_row(df_raw, required_cols=["дата", "сум", "опис", "катег"])
        return pd.read_excel(path, header=header_row)
    else:
        raise ValueError("Поддерживаются только CSV и XLSX.")


def extract_mcc(text: str) -> Optional[int]:
    """Extracts MCC code from description text if present."""
    if not isinstance(text, str):
        return None
    m = re.search(r"mcc[:\s]*([0-9]{4})", text.lower())
    return int(m.group(1)) if m else None


def normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Normalizes raw bank dataframe to standard schema.

    Output columns:
        date, description, amount, bank_category, mcc

    Args:
        df: Raw dataframe.

    Returns:
        Normalized dataframe.

    Raises:
        ValueError: If required columns can't be mapped.
    """
    df = df.copy()
    cols = {c: str(c).lower() for c in df.columns}

    # эвристики маппинга
    date_cols = [c for c, n in cols.items() if "дата" in n or "date" in n]
    desc_cols = [c for c, n in cols.items() if "опис" in n or "назнач" in n or "description" in n]
    amt_cols = [c for c, n in cols.items() if "сум" in n or "amount" in n or "итог" in n]
    cat_cols = [c for c, n in cols.items() if "катег" in n or "category" in n]

    if not date_cols or not desc_cols or not amt_cols:
        raise ValueError("Не удалось определить ключевые колонки: дата/описание/сумма.")

    df = df.rename(columns={
        date_cols[0]: "date",
        desc_cols[0]: "description",
        amt_cols[0]: "amount",
        (cat_cols[0] if cat_cols else None): "bank_category",
    })

    if "bank_category" not in df.columns:
        df["bank_category"] = "Неизвестно"

    df["date"] = pd.to_datetime(df["date"], errors="coerce", dayfirst=True)
    df["amount"] = pd.to_numeric(df["amount"], errors="coerce")
    df["description"] = df["description"].astype(str)
    df["bank_category"] = df["bank_category"].astype(str)

    df = df.dropna(subset=["date", "amount"]).reset_index(drop=True)
    df["mcc"] = df["description"].apply(extract_mcc)

    return df


# ML тип операции: income / transfer / spending

In [58]:
TRANSFER_KWS = [
    "сбп", "перевод", "между счет", "внутрибанков", "пополнение", "копилка",
    "перечисление", "перевод средств", "p2p", "card2card"
]
INCOME_KWS = [
    "зачисление", "зарплат", "salary", "возврат", "refund", "кэшбэк", "cashback",
    "пособие", "стипенд", "пенси", "начисление"
]

REFUND_KWS = ["возврат", "refund", "отмена", "reversal", "сторно"]

EMPLOYER_KWS = [
    "ооо", "зао", "пао", "ип", "ltd", "llc", "inc",
    "работодатель", "salary", "заработ"
]

SELF_TRANSFER_KWS = [
    "между счет", "своими", "себе", "мой счет", "внутрибанков",
    "со счета", "на счет"
]


def weak_label_transaction_type(amount: float, description: str) -> Optional[str]:
    desc = (description or "").lower()

    # 1️⃣ Возвраты
    if any(k in desc for k in REFUND_KWS):
        return "refund"

    # 2️⃣ Внутренние переводы (НЕ доход)
    if any(k in desc for k in SELF_TRANSFER_KWS):
        return "transfer"

    # 3️⃣ Зарплаты, выплаты от организаций
    if any(k in desc for k in EMPLOYER_KWS):
        return "income"

    # 4️⃣ Явные переводы p2p
    if "сбп" in desc or "p2p" in desc:
        if amount > 0:
            return "income"  # входящий p2p = доход
        else:
            return "transfer"

    # 5️⃣ Общие слова перевода
    if "перевод" in desc or "перечисление" in desc:
        if amount > 0:
            return "income"
        else:
            return "transfer"

    # 6️⃣ Spending
    if amount < 0:
        return "spending"

    return None


def train_transaction_type_model(df: pd.DataFrame) -> Pipeline:
    """Trains transaction type classifier.

    Uses weak labels for confident samples only.

    Args:
        df: Normalized dataframe.

    Returns:
        sklearn Pipeline that predicts transaction_type.
    """
    work = df.copy()
    work["weak_type"] = [
        weak_label_transaction_type(a, d) for a, d in zip(work["amount"], work["description"])
    ]
    train_df = work.dropna(subset=["weak_type"]).copy()

    # убираем случаи, где positive amount и transfer — это часто грязные лейблы
    train_df = train_df[
        ~((train_df["weak_type"] == "transfer") & (train_df["amount"] > 0))
    ]

    # Если данных мало — модель будет нестабильна, но всё равно лучше правил.
    X = train_df[["description", "amount"]]
    y = train_df["weak_type"]

    # Признаки: TF-IDF текста + знак суммы
    pre = ColumnTransformer(
        transformers=[
            ("text", TfidfVectorizer(max_features=8000, ngram_range=(1, 2), min_df=2),
             "description"),
            ("num", Pipeline([("scaler", StandardScaler())]), ["amount"]),
        ],
        remainder="drop"
    )

    clf = LogisticRegression(max_iter=2000, class_weight="balanced")
    model = Pipeline([("pre", pre), ("clf", clf)])

    model.fit(X, y)
    return model


def apply_transaction_type_model(df: pd.DataFrame, model: Pipeline) -> pd.DataFrame:
    """Adds transaction_type prediction to dataframe."""
    out = df.copy()
    out["transaction_type"] = model.predict(out[["description", "amount"]])
    return out


# Категоризация расходов

In [59]:
OTHER_BANK_LABELS = {"прочие операции", "прочее", "другое", "неизвестно", ""}


def mcc_to_category(mcc: Optional[int]) -> Optional[str]:
    """Maps MCC to human category if possible."""
    if mcc is None:
        return None
    for cat, codes in MCC_MAP.items():
        if mcc in codes:
            return cat
    return None


def build_pseudo_category(df_spending: pd.DataFrame) -> pd.DataFrame:
    """Builds pseudo labels for category training.

    Args:
        df_spending: Dataframe with only spending transactions.

    Returns:
        Dataframe with new column pseudo_category.
    """
    out = df_spending.copy()
    bank = out["bank_category"].astype(str).str.lower().str.strip()
    out["mcc_category"] = out["mcc"].apply(mcc_to_category)

    out["pseudo_category"] = np.where(
        ~bank.isin(OTHER_BANK_LABELS),
        out["bank_category"],
        np.where(out["mcc_category"].notna(), out["mcc_category"], "Другое")
    )
    return out


def train_spending_category_model(df_spending: pd.DataFrame) -> Pipeline:
    """Trains spending category classifier on pseudo labels.

    Args:
        df_spending: Spending-only dataframe.

    Returns:
        sklearn Pipeline predicting final_category.
    """
    work = build_pseudo_category(df_spending)

    X = work[["description"]]
    y = work["pseudo_category"].astype(str)

    model = Pipeline([
        ("tfidf", TfidfVectorizer(max_features=15000, ngram_range=(1, 2), min_df=2)),
        ("clf", LogisticRegression(max_iter=2000, class_weight="balanced"))
    ])
    model.fit(X["description"], y)
    return model


def apply_spending_category_model(df_spending: pd.DataFrame, model: Pipeline) -> pd.DataFrame:
    """Applies category model and merges with bank categories smartly.

    Policy:
      - If bank category is informative -> keep bank category
      - Else -> use model prediction

    Args:
        df_spending: Spending-only dataframe.
        model: Trained classifier.

    Returns:
        Dataframe with final_category.
    """
    out = df_spending.copy()
    bank = out["bank_category"].astype(str).str.lower().str.strip()

    pred = model.predict(out["description"].astype(str))
    out["ml_category"] = pred

    out["final_category"] = np.where(
        ~bank.isin(OTHER_BANK_LABELS),
        out["bank_category"],
        out["ml_category"]
    )
    return out

# Регулярные платежи: временная периодичность + группировка “мерчанта”

In [60]:
def normalize_merchant_key(text: str) -> str:
    """Normalizes description to stable merchant key.

    Args:
        text: Raw description.

    Returns:
        Normalized merchant key.
    """
    t = (text or "").lower()
    t = re.sub(r"\d+", " ", t)
    t = re.sub(r"[^\w\s]", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    # обрезаем, чтобы не было гигантских ключей
    return t[:80]


@dataclass
class RecurringRule:
    merchant_key: str
    n: int
    avg_amount: float
    total_amount: float
    period_days: float
    period_label: str
    last_date: pd.Timestamp


from sklearn.cluster import DBSCAN


def detect_recurring(df_spending: pd.DataFrame, min_n: int = 3):
    work = df_spending.copy()

    # 1️⃣ Семантический вектор описаний
    tfidf = TfidfVectorizer(
        max_features=2000,
        ngram_range=(1, 2),
        min_df=2
    )
    X_text = tfidf.fit_transform(work["description"].astype(str))

    # 2️⃣ Числовые признаки
    work["day"] = work["date"].dt.day
    work["log_amount"] = np.log1p(work["amount"].abs())

    X_num = work[["day", "log_amount"]].values
    X_num = StandardScaler().fit_transform(X_num)

    # 3️⃣ Объединяем текст + числа
    from scipy.sparse import hstack
    X = hstack([X_text, X_num])

    # 4️⃣ Кластеризация recurring-поведения
    model = DBSCAN(
        eps=0.6,
        min_samples=min_n,
        metric="cosine"
    )

    labels = model.fit_predict(X)
    work["recurring_cluster"] = labels

    rules = []
    for cid in set(labels):
        if cid == -1:
            continue

        g = work[work["recurring_cluster"] == cid].sort_values("date")

        if len(g) < min_n:
            continue

        dates = g["date"].values.astype("datetime64[D]").astype(int)
        intervals = np.diff(dates)

        if len(intervals) == 0:
            continue

        med = float(np.median(intervals))
        mad = float(np.median(np.abs(intervals - med)))

        label = None
        if mad <= 2 and 25 <= med <= 35:
            label = "monthly"
        elif mad <= 2 and 5 <= med <= 9:
            label = "weekly"

        if label is None:
            continue

        rules.append({
            "cluster": cid,
            "example": g["description"].iloc[0][:60],
            "n": len(g),
            "avg_amount": g["amount"].mean(),
            "total_amount": g["amount"].sum(),
            "period": label
        })

    return work, pd.DataFrame(rules)


# Аномалии: контекстный IsolationForest

In [61]:
def detect_contextual_anomalies(df_spending, recurring_merchants, contamination=0.03):
    work = df_spending.copy()
    work["merchant_key"] = work["description"].apply(normalize_merchant_key)

    work["is_recurring"] = work["merchant_key"].isin(recurring_merchants).astype(int)

    work["weekday"] = work["date"].dt.weekday
    work["month"] = work["date"].dt.month
    work["log_abs_amount"] = np.log1p(work["amount"].abs())

    anomalies = []

    for flag in [0, 1]:  # non-recurring and recurring separately
        subset = work[work["is_recurring"] == flag]
        if len(subset) < 10:
            continue

        pre = ColumnTransformer([
            ("num", StandardScaler(), ["log_abs_amount", "weekday", "month"]),
            ("cat", OneHotEncoder(handle_unknown="ignore"), ["final_category"])
        ])

        model = IsolationForest(
            contamination=0.03 if flag == 0 else 0.01,  # подписки строже
            random_state=42
        )

        X = pre.fit_transform(subset[["log_abs_amount", "weekday", "month", "final_category"]])
        preds = model.fit_predict(X)
        scores = model.decision_function(X)

        subset = subset.copy()
        subset["is_anomaly"] = (preds == -1).astype(int)
        subset["anomaly_score"] = scores

        anomalies.append(subset)

    return pd.concat(anomalies)


def filter_low_impact_anomalies(df_anomalies):
    work = df_anomalies.copy()

    medians = (
        work.groupby("final_category")["amount"]
        .apply(lambda x: x.abs().median())
    )

    work["impact"] = work.apply(
        lambda r: abs(r["amount"]) / max(1, medians.get(r["final_category"], 1)),
        axis=1
    )

    # аномалия только если реально влияет на бюджет
    work["is_anomaly"] = np.where(
        (work["is_anomaly"] == 1) & (work["impact"] > 0.5),
        1,
        0
    )

    return work



# Поведенческий baseline + оценка экономии

In [62]:
DEFAULT_SAVING_POTENTIAL = {
    "Фастфуд": 0.35,
    "Рестораны и кафе": 0.30,
    "Такси и каршеринг": 0.30,
    "Связь": 0.15,
    "Маркетплейсы": 0.25,
    "Аптеки": 0.08,
    "Другое": 0.20
}


def monthly_category_spend(df_spending: pd.DataFrame) -> pd.DataFrame:
    """Builds month x category spending matrix.

    Args:
        df_spending: Spending-only dataframe with final_category.

    Returns:
        Pivot table with absolute spending.
    """
    work = df_spending.copy()
    work["month"] = work["date"].dt.to_period("M").astype(str)
    piv = (
        work.groupby(["month", "final_category"])["amount"]
        .sum()
        .abs()
        .unstack(fill_value=0.0)
    )
    return piv


def robust_baseline(piv):
    low = piv.quantile(0.3)
    high = piv.quantile(0.7)

    mask = (piv >= low) & (piv <= high)
    trimmed = piv.where(mask)

    return trimmed.mean(axis=0)


def estimate_savings_plan(
        df_spending: pd.DataFrame,
        recurring_df: pd.DataFrame
) -> Tuple[float, pd.DataFrame, List[str]]:
    """Estimates potential savings based on baseline overspending.

    Args:
        df_spending: Spending-only dataframe.
        recurring_df: Recurring payments summary frame.

    Returns:
        (total_savings, per_category_plan, explanations)
    """
    piv = monthly_category_spend(df_spending)
    if piv.shape[0] < 2:
        return 0.0, pd.DataFrame(), ["Недостаточно месяцев для устойчивой оценки baseline."]

    baseline = robust_baseline(piv)
    overspend = (piv - baseline).clip(lower=0.0)

    # общий перерасход по категориям за все месяцы
    overspend_total = overspend.sum(axis=0).sort_values(ascending=False)

    plan_rows = []
    explanations: List[str] = []

    for cat, extra in overspend_total.items():
        if extra <= 0:
            continue
        coef = DEFAULT_SAVING_POTENTIAL.get(str(cat), DEFAULT_SAVING_POTENTIAL["Другое"])
        save = float(extra * coef)
        plan_rows.append((str(cat), float(extra), coef, save))

    plan = pd.DataFrame(plan_rows, columns=["category", "overspend_total", "saving_coef", "potential_saving"])

    # отдельная аккуратная строка по подпискам/регулярным monthly
    if len(recurring_df) > 0:
        monthly = recurring_df[recurring_df["period_label"] == "monthly"].copy()
        if len(monthly) > 0:
            subs_total = float(monthly["total_amount"].abs().sum())
            # реалистично: 10-30% (переход на тариф, отмена части)
            subs_save = subs_total * 0.20
            explanations.append(
                f"Регулярные monthly-платежи: всего {subs_total:,.0f} ₽, реалистичная оптимизация ~{subs_save:,.0f} ₽."
            )

    total_savings = float(plan["potential_saving"].sum()) if len(plan) else 0.0

    # краткое объяснение топ-точек
    if len(plan) > 0:
        top = plan.sort_values("potential_saving", ascending=False).head(3)
        for _, r in top.iterrows():
            explanations.append(
                f"Категория '{r['category']}': перерасход {r['overspend_total']:,.0f} ₽, "
                f"потенциал экономии ~{r['potential_saving']:,.0f} ₽."
            )

    return round(total_savings, 2), plan.sort_values("potential_saving", ascending=False), explanations


# Отчёт

In [63]:
def format_money(x: float) -> str:
    """Formats money with thousands separator."""
    return f"{x:,.0f} ₽".replace(",", " ")


def build_user_report(
        df_all: pd.DataFrame,
        df_spending: pd.DataFrame,
        recurring_df: pd.DataFrame,
        anomalies_df: pd.DataFrame,
        savings_total: float,
        savings_plan: pd.DataFrame,
        savings_notes: List[str],
        cashflow: Dict[str, float],
        top_n: int = 8
) -> str:
    """Builds Telegram-ready report text.

    Args:
        df_all: All transactions with transaction_type.
        df_spending: Spending-only with final_category.
        recurring_df: Summary of recurring payments.
        anomalies_df: Spending dataframe with anomaly flags.
        savings_total: Total potential savings.
        savings_plan: Per-category plan.
        savings_notes: Explanation bullet points.
        top_n: Number of categories to show.

    Returns:
        Report string.
    """
    lines: List[str] = []

    # период
    d0 = df_all["date"].min().date()
    d1 = df_all["date"].max().date()
    lines.append(f"📅 Период: {d0} — {d1}\n")
    lines.append("💰 Реальный бюджет этой карты:")
    lines.append(f"- Поступило на карту: {format_money(cashflow['inflow'])}")
    lines.append(f"- Потрачено: {format_money(cashflow['spending'])}")
    lines.append(f"- Переведено на другие счета: {format_money(cashflow['transfer_out'])}")
    lines.append("")

    # структура потоков
    flows = df_all.groupby("transaction_type")["amount"].sum()
    lines.append("💳 Денежные потоки (модель типа операций):")
    for k in ["income", "refund", "transfer", "spending"]:
        if k in flows.index:
            lines.append(f"- {k}: {format_money(float(flows[k]))}")
    lines.append("")

    # куда уходят деньги (только spending)
    cat = (
        df_spending.groupby("final_category")["amount"]
        .sum().abs().sort_values(ascending=False)
    )
    total_spend = float(cat.sum()) if len(cat) else 0.0
    lines.append("📊 Куда уходят деньги (только расходы):")
    for c, v in cat.head(top_n).items():
        share = (float(v) / total_spend * 100) if total_spend > 0 else 0
        lines.append(f"- {c}: {format_money(float(v))} ({share:.1f}%)")
    if len(cat) > top_n:
        rest = float(cat.iloc[top_n:].sum())
        lines.append(f"- Другое (остальные): {format_money(rest)}")
    lines.append("")

    # регулярные
    lines.append("🔁 Регулярные платежи (по периодичности):")
    if len(recurring_df) == 0:
        lines.append("- Не найдено устойчивых weekly/monthly шаблонов.")
    else:
        show = recurring_df.sort_values("total_amount").head(10)
        for _, r in show.iterrows():
            lines.append(
                f"- {r['period']} | {r['example']}… "
                f"({int(r['n'])} раз, среднее {format_money(abs(float(r['avg_amount'])))}; "
                f"всего {format_money(abs(float(r['total_amount'])))} )"
            )

    lines.append("")

    # аномалии
    lines.append("⚠️ Нетипичные траты (контекстная модель):")
    anoms = anomalies_df[anomalies_df["is_anomaly"] == 1].copy()
    if len(anoms) == 0:
        lines.append("- Аномалий не обнаружено.")
    else:
        # самые "низкие" anomaly_score
        anoms = anoms.sort_values("anomaly_score").head(8)
        for _, r in anoms.iterrows():
            lines.append(
                f"- {r['date'].date()} | {r['final_category']} | {r['description'][:45]}… → {format_money(abs(float(r['amount'])))}"
            )
    lines.append("")

    # экономия
    lines.append("💰 Потенциал экономии (baseline + план):")
    lines.append(f"- Оценка: ~{format_money(abs(float(savings_total)))} за период.")
    for note in savings_notes[:5]:
        lines.append(f"- {note}")
    if len(savings_plan) > 0:
        lines.append("\nПлан по категориям (топ-5):")
        for _, r in savings_plan.head(5).iterrows():
            lines.append(
                f"- {r['category']}: потенциал ~{format_money(float(r['potential_saving']))} "
                f"(коэф. {int(float(r['saving_coef']) * 100)}%)"
            )

    return "\n".join(lines)


# Итоговый Pipeline

In [64]:
def run_pipeline(path: str) -> Dict[str, object]:
    """Runs full analytics pipeline.

    Args:
        path: Path to bank export file (CSV/XLSX).

    Returns:
        Dict with report and intermediate artifacts.
    """
    # A) load + normalize
    raw = load_bank_file(path)
    df = normalize_columns(raw)

    # B) transaction type model
    type_model = train_transaction_type_model(df)
    df = apply_transaction_type_model(df, type_model)

    df["direction"] = np.where(df["amount"] > 0, "in", "out")

    # Все входящие деньги на эту карту
    inflow = df[df["amount"] > 0]["amount"].sum()

    # Все реальные траты (не переводы)
    spending_out = df[df["transaction_type"] == "spending"]["amount"].abs().sum()

    # Переводы с этой карты наружу
    transfer_out = df[(df["transaction_type"] == "transfer") & (df["amount"] < 0)]["amount"].abs().sum()

    # Реальный доступный бюджет этой карты
    available_budget = inflow

    # Для отчёта
    cashflow = {
        "inflow": float(inflow),
        "spending": float(spending_out),
        "transfer_out": float(transfer_out),
        "available_budget": float(available_budget)
    }

    # C) spending-only
    df_sp = df[df["transaction_type"] == "spending"].copy()
    if len(df_sp) == 0:
        return {"report": "Не найдено операций расходов (spending). Проверьте выгрузку/знак сумм.", "df": df}

    # category model
    cat_model = train_spending_category_model(df_sp)
    df_sp = apply_spending_category_model(df_sp, cat_model)

    # D) recurring (time-based)
    df_sp_keyed, recurring_df = detect_recurring(df_sp, min_n=4)

    # recurring_df теперь уже DataFrame
    if len(recurring_df) > 0:
        recurring_merchants = set(
            df_sp_keyed[df_sp_keyed["recurring_cluster"] != -1]["merchant_key"].astype(str)
        )
    else:
        recurring_merchants = set()


    # E) anomalies (contextual)
    df_an = detect_contextual_anomalies(df_sp_keyed, recurring_merchants=recurring_merchants, contamination=0.03)
    df_an = filter_low_impact_anomalies(df_an)

    # F) savings
    savings_total, savings_plan, savings_notes = estimate_savings_plan(df_sp_keyed, recurring_df)

    # report
    report = build_user_report(
        df_all=df,
        df_spending=df_sp_keyed,
        recurring_df=recurring_df,
        anomalies_df=df_an,
        savings_total=savings_total,
        savings_plan=savings_plan,
        savings_notes=savings_notes,
        cashflow=cashflow
    )

    return {
        "report": report,
        "df_all": df,
        "df_spending": df_sp_keyed,
        "recurring": recurring_df,
        "anomalies": df_an[df_an["is_anomaly"] == 1].copy(),
        "savings_total": savings_total,
        "savings_plan": savings_plan,
        "type_model": type_model,
        "category_model": cat_model,
    }


In [65]:
res = run_pipeline("assets/12 июля 2025 - 11 января 2026.xlsx")
print(res["report"])

📅 Период: 2025-07-13 — 2026-01-11

💰 Реальный бюджет этой карты:
- Поступило на карту: 9 421 ₽
- Потрачено: 66 257 ₽
- Переведено на другие счета: 140 457 ₽

💳 Денежные потоки (модель типа операций):
- income: 9 421 ₽
- transfer: -140 457 ₽
- spending: -66 257 ₽

📊 Куда уходят деньги (только расходы):
- Такси и каршеринг: 28 852 ₽ (43.5%)
- Телефон, интернет, ТВ: 8 206 ₽ (12.4%)
- Фастфуд: 6 316 ₽ (9.5%)
- Другое: 5 401 ₽ (8.2%)
- Транспорт: 5 157 ₽ (7.8%)
- Аптеки: 3 005 ₽ (4.5%)
- Мультимедиа: 1 823 ₽ (2.8%)
- Кафе и рестораны: 1 690 ₽ (2.6%)
- Другое (остальные): 5 807 ₽

🔁 Регулярные платежи (по периодичности):
- Не найдено устойчивых weekly/monthly шаблонов.

⚠️ Нетипичные траты (контекстная модель):
- Аномалий не обнаружено.

💰 Потенциал экономии (baseline + план):
- Оценка: ~10 526 ₽ за период.
- Категория 'Такси и каршеринг': перерасход 22,770 ₽, потенциал экономии ~6,831 ₽.
- Категория 'Фастфуд': перерасход 3,337 ₽, потенциал экономии ~1,168 ₽.
- Категория 'Кафе и рестораны': 